-This code merges all the csv files in the folder "BTC_6_Months_Data" which is stored in my computer.
-Change this path "D:\BTC_6_Months_Data" to the folder path where you have stored all the csv files you want to merge 
-I have saved the merged_data as a csv file you can save it as any other file type , to save space as csv takes a lot of space on the computer

In [6]:
import pandas as pd
import os

# Path to your folder containing the CSVs
folder_path = r"D:\BTC_6_Months_Data"

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# Read and concatenate all CSVs
df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]
merged_df = pd.concat(df_list, ignore_index=True)

# Save to a new CSV
merged_df.to_csv("merged_data.csv", index=False)
merged_df

print(" Merged CSV saved as merged_data.csv")


 Merged CSV saved as merged_data.csv


-This code extracts unique symbols from the dataframe

In [ ]:
#This code is not nessarly but it is useful for later

# Get unique symbols
unique_symbols = merged_df["product_symbol"].unique()

# Convert to a Python list if needed
unique_symbols_list = unique_symbols.tolist()

print("Unique Symbols:")
print(unique_symbols_list)
print(len(unique_symbols_list), "unique symbols found.")


Unique Symbols:
['C-BTC-94800-010125', 'C-BTC-94000-010125', 'C-BTC-111000-030125', 'C-BTC-93400-010125', 'P-BTC-92000-010125', 'C-BTC-94400-010125', 'P-BTC-92800-010125', 'P-BTC-91600-010125', 'C-BTC-96000-010125', 'C-BTC-95000-010125', 'P-BTC-93400-010125', 'P-BTC-94600-010125', 'C-BTC-93600-010125', 'C-BTC-93200-010125', 'P-BTC-92600-010125', 'C-BTC-94600-010125', 'P-BTC-88400-010125', 'P-BTC-90800-010125', 'P-BTC-94000-010125', 'P-BTC-93600-020125', 'P-BTC-94800-010125', 'C-BTC-98500-030125', 'C-BTC-99200-010125', 'C-BTC-93800-010125', 'C-BTC-97000-010125', 'P-BTC-86000-100125', 'C-BTC-97600-010125', 'C-BTC-113000-030125', 'C-BTC-96200-010125', 'C-BTC-96600-010125', 'C-BTC-98400-020125', 'C-BTC-98000-030125', 'P-BTC-96000-030125', 'C-BTC-96000-030125', 'P-BTC-94000-030125', 'C-BTC-96400-010125', 'P-BTC-90400-010125', 'C-BTC-97200-010125', 'C-BTC-95200-010125', 'C-BTC-94000-030125', 'C-BTC-98800-010125', 'C-BTC-99600-020125', 'C-BTC-98000-010125', 'C-BTC-97400-010125', 'P-BTC-93800-

In [14]:
print(merged_df.columns)

Index(['product_symbol', 'price', 'size', 'timestamp', 'buyer_role'], dtype='object')


-The code below is the code to convert tick data to OHLCV data and save it to parquet file (lot smaller than a csv file) 

In [5]:


# Assume merged_df has columns: ["timestamp", "price", "size", "product_symbol"]
# Convert timestamp if needed
merged_df["timestamp"] = pd.to_datetime(merged_df["timestamp"] , errors="coerce")
merged_df = merged_df.set_index("timestamp")

# Vectorized groupby-resample → OHLCV for each symbol
final_bars = (
    merged_df
    .groupby("product_symbol")
    .resample("1H")
    .agg(
        open=("price", "first"),
        high=("price", "max"),
        low=("price", "min"),
        close=("price", "last"),
        volume=("size", "sum")
    )
    .reset_index()
)

# Save result
final_bars.to_parquet("all_symbols_1h_bars.parquet", compression="snappy")

print(" 1H candlesticks generated for all symbols → all_symbols_1h_bars.csv")


C:\Users\Siddhi\AppData\Local\Temp\ipykernel_14228\2821240112.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample("1H")


 1H candlesticks generated for all symbols → all_symbols_1h_bars.csv


In [9]:
final_bars

,product_symbol,timestamp,open,high,low,close,volume
0,C-BTC-100000-010225,2025-01-30 04:00:00,5445.0,5445.0,5445.0,5445.0,8.0
1,C-BTC-100000-010225,2025-01-30 05:00:00,NaN,NaN,NaN,NaN,0.0
2,C-BTC-100000-010225,2025-01-30 06:00:00,NaN,NaN,NaN,NaN,0.0
3,C-BTC-100000-010225,2025-01-30 07:00:00,NaN,NaN,NaN,NaN,0.0
4,C-BTC-100000-010225,2025-01-30 08:00:00,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...
2041124,P-BTC-99800-300125,2025-01-30 07:00:00,1.0,1.0,1.0,1.0,10.0
2041125,P-BTC-99800-300125,2025-01-30 08:00:00,NaN,NaN,NaN,NaN,0.0
2041126,P-BTC-99800-300125,2025-01-30 09:00:00,NaN,NaN,NaN,NaN,0.0
2041127,P-BTC-99800-300125,2025-01-30 10:00:00,0.2,0.3,0.2,0.3,120.0


-This code makes a dictionary with symbol as key and dataframe as the value

In [ ]:

merged_df["timestamp"] = pd.to_datetime(
    merged_df["timestamp"], errors="coerce"
)
merged_df = merged_df.set_index("timestamp")

symbol_dict = {
    symbol: df for symbol, df in merged_df.groupby("product_symbol")
}
symbol_dict


{'C-BTC-100000-010225':                                  product_symbol        price   size buyer_role
 timestamp                                                                     
 2025-01-30 04:24:07.302286  C-BTC-100000-010225  5445.000000    8.0      maker
 2025-01-30 09:55:12.254769  C-BTC-100000-010225  5709.000000    8.0      taker
 2025-01-30 23:52:30.477881  C-BTC-100000-010225  4990.000000    1.0      taker
 2025-01-31 03:07:01.157049  C-BTC-100000-010225  4240.000000    5.0      taker
 2025-01-31 07:24:51.578664  C-BTC-100000-010225  4963.000000    5.0      taker
 2025-01-31 08:31:23.041844  C-BTC-100000-010225  4170.000000    1.0      maker
 2025-01-31 11:11:03.833533  C-BTC-100000-010225  4753.000000    1.0      taker
 2025-01-31 11:26:14.744437  C-BTC-100000-010225  4741.000000    5.0      maker
 2025-01-31 14:41:55.514714  C-BTC-100000-010225  4558.000000    5.0      taker
 2025-01-31 15:23:15.949201  C-BTC-100000-010225  5000.000000    2.0      taker
 2025-01-31 17:42

In [3]:
import polars as pl

# Define the correct column names (adjust to match your CSV)
columns = ["product_symbol", "price", "size", "timestamp", "side"]

df = pl.read_csv(
    "merged_data.csv",
    has_header=False,   # treat first row as data, not header
    new_columns=columns # assign correct names
)

# Convert timestamp to datetime
df = df.with_columns(
    pl.col("timestamp").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S%.f", strict=False)
)

# Sort the DataFrame by product_symbol and timestamp (ascending order)
# Filter out rows with null values in critical columns
df = df.filter(
    pl.col("timestamp").is_not_null() & 
    pl.col("product_symbol").is_not_null() &
    pl.col("price").is_not_null()
)


# Now resample into 1-minute OHLCV bars using groupby_dynamic
bars = (
    df.lazy()
    .group_by_dynamic(
        index_column="timestamp",
        every="1m",
        group_by="product_symbol"  # Using the new parameter name
    )
    .agg([
        pl.col("price").first().alias("open"),
        pl.col("price").max().alias("high"),
        pl.col("price").min().alias("low"),
        pl.col("price").last().alias("close"),
        pl.col("size").sum().alias("volume"),
    ])
    .collect()  # Execute the query
)

print(bars)

shape: (4_296_864, 7)
┌─────────────────────┬─────────────────────┬─────────┬─────────┬─────────┬─────────┬────────┐
│ product_symbol      ┆ timestamp           ┆ open    ┆ high    ┆ low     ┆ close   ┆ volume │
│ ---                 ┆ ---                 ┆ ---     ┆ ---     ┆ ---     ┆ ---     ┆ ---    │
│ str                 ┆ datetime[μs]        ┆ str     ┆ str     ┆ str     ┆ str     ┆ str    │
╞═════════════════════╪═════════════════════╪═════════╪═════════╪═════════╪═════════╪════════╡
│ C-BTC-62000-300525  ┆ 2025-04-07 14:49:00 ┆ 18313.0 ┆ 18313.0 ┆ 18313.0 ┆ 18313.0 ┆ null   │
│ C-BTC-62000-300525  ┆ 2025-04-07 16:04:00 ┆ 18269.0 ┆ 18269.0 ┆ 18269.0 ┆ 18269.0 ┆ null   │
│ C-BTC-62000-300525  ┆ 2025-04-09 17:46:00 ┆ 21029.0 ┆ 21029.0 ┆ 21029.0 ┆ 21029.0 ┆ null   │
│ C-BTC-62000-300525  ┆ 2025-04-09 21:13:00 ┆ 22446.0 ┆ 22446.0 ┆ 22446.0 ┆ 22446.0 ┆ null   │
│ C-BTC-62000-300525  ┆ 2025-04-09 21:14:00 ┆ 22816.0 ┆ 22816.0 ┆ 22816.0 ┆ 22816.0 ┆ null   │
│ …                   ┆ …   